In [1]:
# First we want to set the stage and import all required packages as well as the data to perform the analysis.
# The data is imported efficiently, loading all data sets in a dictonary where each key (geography)
# holds a list of dataframes with all route information within that geography split by state.
# Simultaneously the names of each geography gets extracted from the folder names and the names of each state
# from the file names.

# Load the required packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import glob

# Collect folder paths of the data
all_dir = "G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-usa-routes-aug-2020"
ca_dir = "G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-routes-ca"
midwest_dir = "G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-routes-midwest"
northeast_dir = "G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-routes-northeast"
southeast_dir = "G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-routes-southeast"
westcoast_dir = "G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-routes-westcoast"
m1_dir = "G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-routes-mountains1"
m2_dir = "G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-routes-mountain2"
ms2_dir = "G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-routes-mountains2"

# Extract and save the data found in all_dir that holds all route information of the USA from 2020 and save it to "all".
all = pd.read_json("G:/My Drive/Trainings/Python/Trainings_file/climbing project/openbeta-usa-routes-aug-2020/openbeta-usa-routes-aug-2020.jsonlines", lines= True)


# Create a list of all folder path strings, excluding all_dir
folder_paths = [ca_dir, midwest_dir, northeast_dir, southeast_dir, westcoast_dir, m1_dir, m2_dir, ms2_dir]

# Extract the other geographies:

geographies_data = {}  # Initialize an empty dictionary to store the resulting list of dataframes for each geography
geographies_states = {} # Initialize an empty dictionary to store the state name strings in lists for each geography

# Here we fill the dictinary geography with a list of dataframes for each geograpy. Each dataframe corresponds to a US state:

for folder_path in folder_paths:
    file_list = glob.glob(folder_path + '/*.jsonlines')  # Get a list of all JSON files in the folder
    file_list.sort() # Sort the list aphabetically, since glob does not do it

    # Extract the state names from the file names in each folder.
    file_list_trunc = [] # Initialize empty list
    for file in file_list : # For each file in the file_list, truncate the string to only keep the state names
        file_trunc = file.split('-routes.jsonline', 1)[0]
        file_trunc = file_trunc.split('\\', 1)[-1]
        file_list_trunc.append(file_trunc) # Append the state name to the list of statenames

    folder_data_frames = []  # List to store the dataframes for each geography

    for file_name in file_list:
        data_frame = pd.read_json(file_name, lines = True)
        folder_data_frames.append(data_frame)

        folder_name = os.path.basename(folder_path) # Extract the folder name
        # Strip the first part of the folder name, only keeping the last element of the split.
        folder_name = folder_name.split('routes-', 1)[-1] 
        
    # Add the list of DataFrames and state names to the dictionary with the folder name as the key.
    # The names need to be unique for later variable extraction, so we add _data or _state to the string.
    
    geographies_data[folder_name + "_data"] = folder_data_frames
    geographies_states[folder_name + "_state"] = file_list_trunc

In [2]:
# Having the data imported, we now want to wrangle the data. 
# This code chunk saves each geography in its own variable and assigns key:value pairs to the dataframes for each state
# Afterwards each state will get its own variable with all the route information from that state.


# look at the keys generated for the two dictionary
print(geographies_data.keys())

print(geographies_states.keys())

# Extract the keys from the dictionaries as variables with the list values defining those variables.
# Save the list of keys in a list.

list_of_state_names = []
list_of_dataframes = []

for key, value in geographies_data.items(): # geographies_data.items() provides a way to access and work with the individual key-value pairs of the dictionary.
    globals()[key] = value 
    # This line dynamically creates a new variable within the loop to add to the global namespace with the name of the current key, 
    # and assigns the corresponding value.
    list_of_dataframes.append(globals()[key]) # fill this list with the dataframes.

for key, value in geographies_states.items():
    globals()[key] = value
    list_of_state_names.append(globals()[key]) 

# After having each region as a variable with a list of dataframes or a list of their state names,
# we want to combine them. Each state name should become a variable for its corresponding data frame.
# States that appear as duplicates are tracked and saved seperately.

# to track the count of each state name across region and to avoid overwriting data frames of states that already appeared in another geopgraphy,
# we need to keep track of their counts. This helps control duplicates.

state_counts = {} # initialize dictionary to track state counts. 
duplicates = {} # initialize dictionary to track duplicate states. 
list_of_all_states_and_data = {} # initialze dictionary without duplicates, collecting the first occurance of each state.

# Here we create a nested for-loop that iterates over the lists containing the state names and dataframes.
# Zip allows for iteration at the same time. For each item of the lists we have lists.
# The nested for loop therefore iterates over the items of the lists lists.

for states, dataframes in zip(list_of_state_names, list_of_dataframes) :
    for state, dataframe in zip(states, dataframes) :
        # Add state to the global namespace if it does not exist so far and set its count to 1.
        if state not in globals():
            globals()[state] = dataframe
            state_counts[state] = 1
            list_of_all_states_and_data[state] = dataframe
        else:
        # If state already exists, save the duplicate in a sperate variable, increase its count by 1 and add the count value to the variable name.   
            count = state_counts.get(state, 1) + 1
            state_counts[state] = count
            state_i = state + "_" + str(count)
            globals()[state_i] = dataframe
            duplicates[state] = count


dict_keys(['ca_data', 'midwest_data', 'northeast_data', 'southeast_data', 'westcoast_data', 'mountains1_data', 'mountain2_data', 'mountains2_data'])
dict_keys(['ca_state', 'midwest_state', 'northeast_state', 'southeast_state', 'westcoast_state', 'mountains1_state', 'mountain2_state', 'mountains2_state'])


In [3]:
# This code chunk compares the duplicated state dfs and joins their data frames if they are not equal without duplication.
# If the data is equal, we ignore the second data set of that state.
# This code chunk then adds a State column to each data frame containing the state name, 
# to later identify from which state that route is coming from. 
# We then merge all state dfs into a single df.

# First we check if data frames of state duplicates are identical.

print(duplicates)

are_equal = mi.equals(mi_2)

print(are_equal)

# Looking at the namespace and size (rows) of all duplicate data frames,
# it is fair to assume, that all data frames in the list "duplicates" are actually duplicates.
# They can therefore be ignored in further analysis and we continue with the list of all states filled earlier.


len(list_of_all_states_and_data) # How many states

# Add the key (state) as a separate column to each DataFrame
for state, df in list_of_all_states_and_data.items():
    df['State'] = state

# Merge all dfs of the individual states into a single df containing all routes
df_all_states = pd.concat(list_of_all_states_and_data.values(), ignore_index=True)



{'mi': 2, 'az': 2, 'id': 2, 'mt': 2, 'nm': 2, 'nv': 2, 'wy': 2}
True


In [4]:
print(df_all_states)

                 route_name  \
0                Gravel Pit   
1            Random Impulse   
2             The Tick Wall   
3              Orange Crush   
4       Wimovi Wonder Winos   
...                     ...   
209803  Running Out of Blow   
209804          Red Nations   
209805         Planet Earth   
209806       Heart of Stone   
209807       Crevasse Route   

                                                    grade safety  \
0       {'YDS': '5.12b/c', 'French': '7b+', 'Ewbanks':...          
1       {'YDS': '5.7', 'French': '5a', 'Ewbanks': '15'...          
2                          {'YDS': 'V-easy', 'Font': '3'}          
3       {'YDS': '5.11b/c', 'French': '6c+', 'Ewbanks':...          
4       {'YDS': '5.10-', 'French': '6a', 'Ewbanks': '1...          
...                                                   ...    ...   
209803  {'YDS': '5.6', 'French': '4c', 'Ewbanks': '14'...          
209804  {'YDS': '5.11c', 'French': '6c+', 'Ewbanks': '...          
209805  {'YDS':

In [7]:
# This code chunk compares the two data frames, the published data frame containing all data from August 2020
# and our generated df of all routes from all states.

# Make sure there are no duplicates of routes
all.drop_duplicates(subset = 'route_name', inplace = True)
df_all_states.drop_duplicates(subset = 'route_name', inplace = True)
all.reset_index(drop=True, inplace=True)
df_all_states.reset_index(drop=True, inplace=True)

# Compare the length and the columns of the two dfs.
print(len(all))
print(all.columns)
print(len(df_all_states))
print(df_all_states.columns)

# For our analysis of climbing grades in the US, we are less intersted in information like First ascent or metadata.
# We can therefore drop those columns.
 
all.drop(columns=['fa', 'metadata', 'protection', 'safety', 'description', 'location'], inplace = True)
df_all_states.drop(columns=['fa', 'metadata', 'protection', 'safety', 'description', 'location'], inplace = True)



all_list_route_names = all['route_name'].tolist()
df_all_states_list_route_names = df_all_states['route_name'].tolist()
print(all_list_route_names[0:5])
print(df_all_states_list_route_names[0:5])

# Here we want to figure out which routes are different between the two sets. 
# Since looping if statements over 200K long lists is not efficient we use the set data type which has 
# better performance for large datasets because they leverage hashing and indexing.

set_all_routes = set(all_list_route_names)
set_df_all_states_routes = set(df_all_states_list_route_names)

all_unique_routes = list(set_all_routes - set_df_all_states_routes)
df_all_states_unique_routes = list(set_df_all_states_routes - set_all_routes)

print(all_unique_routes[0:20])
print(df_all_states_unique_routes[0:20])

print(len(all_unique_routes))
print(len(df_all_states_unique_routes))

perc_unique_all = len(all_unique_routes)/len(all)*100
perc_unique_df_all_states = len(df_all_states_unique_routes)/len(df_all_states)*100

print(perc_unique_all)
print(perc_unique_df_all_states)

# 1.95% of routes in "all" are unique for "all" while 13.7 % of routes are unique for the "df_all_states".
# The more complete data set seems to be "df_all_states" while it also contains state information which we want to analyze and compare.
# "df_all_states" contains unique columns holding the Mountain project sector and route id. If the "all" df would have the column "mp_sector_id" too,
# we could join some of the data via sector_ids.
# This is not the case. 
# For our further analysis we will therefore focus on the "df_all_states" dataset.


136632
Index(['route_name', 'grade', 'type'], dtype='object')
155224
Index(['route_name', 'grade', 'type', 'mp_sector_id', 'mp_route_id', 'State'], dtype='object')


KeyError: "['fa', 'metadata', 'protection', 'safety', 'description', 'location'] not found in axis"

In [17]:
# This code chunk will further clean the data frame to prepare the final dataset before we jump into the analysis.

# Here we convert a list of dictionaries into a detaframe with the keys as column headers.
grades_df = pd.DataFrame.from_records(df_all_states['grade'])

# Add a suffix to the column names and drop grading systems we are not interested in.
modified_columns = [col + '_grading' for col in grades_df.columns]
grades_df.columns = modified_columns
grades_df.drop(columns=['Font_grading', 'yds_aid_grading'], inplace = True)

# merge the grades_df with df_all_states.
data_final = pd.concat([df_all_states, grades_df], axis = 1)
data_final.drop(columns=['grade'], inplace = True)

# Make the 'type' data column easier to use
data_final['type'] = data_final['type'].astype(str)
data_final['type'] = data_final['type'].str.replace("{'", "")
data_final['type'] = data_final['type'].str.replace("': True}", "")


# Filter for only sport routes
sport_data = data_final[data_final['type'] == 'sport']
sport_data = sport_data.reset_index()
print(sport_data)

        index                     route_name   type  mp_sector_id  \
0           3                   Orange Crush  sport   105817198.0   
1           4            Wimovi Wonder Winos  sport   113627837.0   
2         236              Watch Tower Arete  sport   118261975.0   
3         237           Violent Pacification  sport   117330322.0   
4         239                        Y'ain't  sport   110143719.0   
...       ...                            ...    ...           ...   
40471  155212            Bitch Muscle direct  sport           NaN   
40472  155213                   Bitch Muscle  sport           NaN   
40473  155214                    Belly Crawl  sport           NaN   
40474  155215          A Place to Meet Woody  sport           NaN   
40475  155216  A Chance 4 the Tennessee Stud  sport           NaN   

       mp_route_id State YDS_grading French_grading Ewbanks_grading  \
0      105817201.0    ca     5.11b/c            6c+              23   
1      118979787.0    ca     